In [5]:
import sys
import os
import json
import pandas as pd
from dotenv import load_dotenv
from tqdm.auto import tqdm

# --- Step 1: モジュールをインポートするためのパス設定 ---
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', 'src')))

try:
    from scopus_handler import ScopusSearcher
except ImportError:
    print("❌ Error: 'src/scopus_handler.py' が見つかりません。")
    raise

# --- Step 2: 設定 ---
load_dotenv()
SCOPUS_API_KEY = os.getenv("SCOPUS_API_KEY")
if not SCOPUS_API_KEY:
    raise ValueError("SCOPUS_API_KEYが設定されていません。")

# --- 出力先の定義 ---
OUTPUT_DIR = "../data/datapapers/raw/"
OUTPUT_FILE = os.path.join(OUTPUT_DIR, "all_datapapers_raw.json")
os.makedirs(OUTPUT_DIR, exist_ok=True)

# --- Step 3: データ論文リストの収集 ---

def collect_and_save_all_data_papers():
    """Scopusからデータ論文を全件収集し、フィルタリングしてJSONとして保存する"""
    
    # 既にファイルが存在する場合は処理を中断
    if os.path.exists(OUTPUT_FILE):
        print(f"ℹ️ Raw data file already exists at {OUTPUT_FILE}. Skipping.")
        # ファイルの中身を確認して件数を表示
        with open(OUTPUT_FILE, 'r', encoding='utf-8') as f:
            existing_data = json.load(f)
        print(f"   Existing file contains {len(existing_data)} papers.")
        return

    print("--- Collecting all data papers from Scopus ---")
    scopus = ScopusSearcher(api_key=SCOPUS_API_KEY)
    
    query = "DOCTYPE(dp)"
    # 上限なしで全件取得（float('inf')は無限大を意味します）
    all_data_papers = scopus.fetch_all_results(query, max_results=float('inf'))
    
    if not all_data_papers:
        print("❌ No data papers found or an error occurred.")
        return
    
    print(f"\n✅ Successfully collected {len(all_data_papers):,} total data papers.")
    
    # --- Step 4: 被引用数でフィルタリング ---
    print("--- Filtering papers with citedby-count >= 2 ---")
    
    filtered_papers = []
    for paper in tqdm(all_data_papers, desc="Filtering"):
        try:
            # citedby-countを整数に変換して比較
            cite_count = int(paper.get('citedby-count', 0))
            if cite_count >= 2:
                filtered_papers.append(paper)
        except (ValueError, TypeError):
            continue # 変換できない場合はスキップ

    print(f"Found {len(filtered_papers):,} papers meeting the criteria.")
    
    # --- Step 5: 結果をJSONファイルとして保存 ---
    with open(OUTPUT_FILE, 'w', encoding='utf-8') as f:
        json.dump(filtered_papers, f, ensure_ascii=False, indent=4)
        
    print(f"✨ Successfully saved filtered data to {OUTPUT_FILE}")

# --- 実行 ---
if __name__ == '__main__':
    collect_and_save_all_data_papers()

--- Collecting all data papers from Scopus ---


Searching Scopus: 100%|██████████| 19813/19813 [25:43<00:00, 12.84 papers/s]  



✅ Successfully collected 19,813 total data papers.
--- Filtering papers with citedby-count >= 2 ---


Filtering: 100%|██████████| 19813/19813 [00:00<00:00, 2203238.38it/s]


Found 13,339 papers meeting the criteria.
✨ Successfully saved filtered data to ../data/datapapers/raw/all_datapapers_raw.json
